In [16]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import backtrader.indicators as btind
import yfinance as yf
import pandas as pd

# --- MACD 策略 ---
class MACDCrossoverFull(bt.Strategy):
    params = (('fast_period', 12), ('slow_period', 26), ('signal_period', 9))

    def __init__(self):
        self.pending_buy = False     # 昨天決定今天買
        self.pending_sell = False    # 昨天決定今天賣
        self.order = None

        self.macd = btind.MACD(period_me1=self.p.fast_period,
                               period_me2=self.p.slow_period,
                               period_signal=self.p.signal_period)
        self.crossover = btind.CrossOver(self.macd.macd, self.macd.signal)

    def notify_order(self, order):
        # Submitted / Accepted → 代表掛單成功，但尚未成交
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            action = "買入" if order.isbuy() else "賣出"
            print(f"{self.data.datetime.date(0)} 📌 {action}成交！"
                  f" 成交價: {order.executed.price:.2f}, "
                  f"股數: {order.executed.size}, "
                  f"現金剩餘: {self.broker.getcash():.2f}")
            self.order = None  # 解鎖

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f"{self.data.datetime.date(0)} ⚠️ 訂單失敗（{order.getstatusname()}），已解鎖")
            self.order = None  # 失敗也要解鎖！

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"{self.data.datetime.date(0)} 🤝 交易平倉：毛利 {trade.pnl:.2f}, 淨利 {trade.pnlcomm:.2f}")

    def next(self):
        # 1️⃣ 若還有訂單等待成交 → 不做事
        if self.order:
            return

        # 2️⃣ 先執行前一天決定的交易 → 用今天開盤價成交
        # 賣在先（避免同一天又買又賣）
        if self.pending_sell and self.position:
            size = self.position.size
            self.order = self.sell(size=size)
            print(f"{self.data.datetime.date(0)} ⛔️ 全部賣出（執行昨日死叉）"
                  f" 開盤價: {self.data.open[0]:.2f}, 股數: {size}")
            self.pending_sell = False
            return  # 今日已執行，退出

        # 再處理買
        if self.pending_buy and not self.position:
            cash = self.broker.getcash()
            open_price = self.data.open[0]
            size = int(cash // open_price)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進（執行昨日金叉）"
                      f" 開盤價: {open_price:.2f}, 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足無法買進")

            self.pending_buy = False
            return

        # 3️⃣ 使用今日收盤資料判斷明日行動
        if self.crossover[0] > 0:
            self.pending_buy = True     # 金叉 → 明天買
            self.pending_sell = False
            # print(f"{self.data.datetime.date(0)} ▶️ 金叉出現，預約明天買")
        elif self.crossover[0] < 0:
            self.pending_sell = True    # 死叉 → 明天賣
            self.pending_buy = False
            # print(f"{self.data.datetime.date(0)} ◀️ 死叉出現，預約明天賣")



# --- Buy & Hold 策略 (與 Sizer 配合) ---
class BuyAndHoldFull(bt.Strategy):
    def __init__(self):
        self.buy_signal = False  # 當天觸發購買信號
        self.order = None

    def next(self):
        # 如果還沒買，觸發購買信號
        if not self.position and not self.buy_signal:
            self.buy_signal = True
            return

        # 前一天觸發 buy_signal → 今天開盤執行
        if self.buy_signal and not self.position:
            open_price = self.data.open[0]
            cash = self.broker.getcash()

            size = int(cash // open_price)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進！價格: {open_price:.2f}, 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足，無法買入")

            self.buy_signal = False

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"{self.data.datetime.date(0)} 📌 成交！成交價: {order.executed.price:.2f}, "
                      f"股數: {order.executed.size}")

# MACD + 布林通道策略                
class MACD_BB_Full(bt.Strategy):
    params = dict(
        fast_period=12,
        slow_period=26,
        signal_period=9,
        bb_period=20,
        bb_dev=2,
    )

    def __init__(self):
        self.order = None
        self.pending_buy = False
        self.pending_sell = False

        # MACD 指標
        self.macd = btind.MACD(
            self.data,
            period_me1=self.p.fast_period,
            period_me2=self.p.slow_period,
            period_signal=self.p.signal_period
        )
        self.crossover = btind.CrossOver(self.macd.macd, self.macd.signal)

        # 布林通道
        self.bbands = btind.BollingerBands(
            self.data.close,
            period=self.p.bb_period,
            devfactor=self.p.bb_dev
        )

    # 訂單狀態紀錄
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            action = "買入" if order.isbuy() else "賣出"
            print(f"{self.data.datetime.date(0)} 📌 {action}成交！"
                  f" 成交價: {order.executed.price:.2f}, "
                  f"股數: {order.executed.size}, "
                  f"現金剩餘: {self.broker.getcash():.2f}")
            self.order = None

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f"{self.data.datetime.date(0)} ⚠️ 訂單失敗（{order.getstatusname()}）")
            self.order = None

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"{self.data.datetime.date(0)} 🤝 平倉：毛利 {trade.pnl:.2f}")

    def next(self):
        if self.order:
            return

        close = self.data.close[0]
        openp = self.data.open[0]
        mid = self.bbands.mid[0]
        top = self.bbands.top[0]
        bot = self.bbands.bot[0]

        # === Step1: 先執行昨天決定的動作 (用今天開盤價) ===
        if self.pending_sell and self.position:
            size = self.position.size
            self.order = self.sell(size=size)
            print(f"{self.data.datetime.date(0)} ⛔️ 全部賣出（執行昨日死叉） 開盤: {openp:.2f}")
            self.pending_sell = False
            return

        if self.pending_buy and not self.position:
            cash = self.broker.getcash()
            size = int(cash // openp)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進（執行昨日金叉+布林）"
                      f" 開盤: {openp:.2f} 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足無法買進")
            self.pending_buy = False
            return

        # === Step2: 用今天收盤判斷明天行動 ===
        if self.crossover[0] > 0:  # 金叉 → 偏多
            if close <= mid:  # 價格在布林通道中軌下側才買
                self.pending_buy = True
                self.pending_sell = False

        elif self.crossover[0] < 0:  # 死叉 → 偏空
            self.pending_sell = True
            self.pending_buy = False




# --- 下載資料 (加上 auto_adjust=True 確保使用調整後價格) ---
STOCK_TICKER = '2330.TW'
BENCH_TICKER = '2330.TW'
START = '2015-01-01'
END = '2024-12-30'

# **重要修正：加上 auto_adjust=True 確保使用調整後價格**
df2330 = yf.download(STOCK_TICKER, start=START, end=END, auto_adjust=True)
df0050 = yf.download(BENCH_TICKER, start=START, end=END, auto_adjust=True)

# 如果 auto_adjust=True，通常不會有多級索引，但保留程式碼以防萬一
if isinstance(df2330.columns, pd.MultiIndex):
    df2330.columns = df2330.columns.droplevel(1)

if isinstance(df0050.columns, pd.MultiIndex):
    df0050.columns = df0050.columns.droplevel(1)


# --- 回測 MACD 策略 ---
data1 = bt.feeds.PandasData(dataname=df2330)
cerebro1 = bt.Cerebro()
cerebro1.addstrategy(MACDCrossoverFull)
cerebro1.adddata(data1)
cerebro1.broker.setcash(100000)
print("\n🚀 回測 MACD 策略中 ...")
cerebro1.run()
macd_final_value = cerebro1.broker.getvalue()
print("===========================")
# --- 回測 Buy & Hold 策略 ---
data2 = bt.feeds.PandasData(dataname=df0050)
cerebro2 = bt.Cerebro()
cerebro2.addsizer(bt.sizers.FixedSize, stake=100)
cerebro2.addstrategy(BuyAndHoldFull)
cerebro2.adddata(data2)
cerebro2.broker.setcash(100000)
print("\n📌 回測 Buy & Hold 策略中 ...")
cerebro2.run()
bh_final_value = cerebro2.broker.getvalue()
# --- 回測 MACD + 布林通道策略 ---
data3 = bt.feeds.PandasData(dataname=df2330)
cerebro3 = bt.Cerebro()
cerebro3.addstrategy(MACD_BB_Full)
cerebro3.adddata(data3)
cerebro3.broker.setcash(100000)
print("\n📌 回測 MACD + 布林通道策略中 ...")
cerebro3.run()
bb_final_value = cerebro3.broker.getvalue()

# --- 結果輸出 ---
print("\n===========================")
print("📊 回測結果比較")
print(f"期間: {START} 至 {END}")
print("---------------------------")
print(f"MACD ({STOCK_TICKER}) 最終資金: {macd_final_value:.2f}  ─ 報酬率: {(macd_final_value/100000-1):.2%}")
print(f"B&H ({BENCH_TICKER}) 最終資金: {bh_final_value:.2f} ─ 報酬率: {(bh_final_value/100000-1):.2%}")
print(f"MACD + BB ({STOCK_TICKER}) 最終資金: {bb_final_value:.2f} ─ 報酬率: {(bb_final_value/100000-1):.2%}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



🚀 回測 MACD 策略中 ...
2015-03-20 🚀 全倉買進（執行昨日金叉） 開盤價: 111.83, 股數: 894
2015-03-23 ⚠️ 訂單失敗（Margin），已解鎖
2015-04-27 🚀 全倉買進（執行昨日金叉） 開盤價: 112.94, 股數: 885
2015-04-28 ⚠️ 訂單失敗（Margin），已解鎖
2015-05-12 🚀 全倉買進（執行昨日金叉） 開盤價: 108.51, 股數: 921
2015-05-13 ⚠️ 訂單失敗（Margin），已解鎖
2015-06-16 🚀 全倉買進（執行昨日金叉） 開盤價: 105.56, 股數: 947
2015-06-17 📌 買入成交！ 成交價: 105.56, 股數: 947, 現金剩餘: 38.81
2015-07-10 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 103.96, 股數: 947
2015-07-13 📌 賣出成交！ 成交價: 104.34, 股數: -947, 現金剩餘: 98851.32
2015-07-13 🤝 交易平倉：毛利 -1148.68, 淨利 -1148.68
2015-07-22 🚀 全倉買進（執行昨日金叉） 開盤價: 105.87, 股數: 933
2015-07-23 📌 買入成交！ 成交價: 104.34, 股數: 933, 現金剩餘: 1499.59
2015-07-24 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 105.10, 股數: 933
2015-07-27 📌 賣出成交！ 成交價: 102.82, 股數: -933, 現金剩餘: 97430.14
2015-07-27 🤝 交易平倉：毛利 -1421.18, 淨利 -1421.18
2015-08-03 🚀 全倉買進（執行昨日金叉） 開盤價: 105.49, 股數: 923
2015-08-04 📌 買入成交！ 成交價: 102.82, 股數: 923, 現金剩餘: 2527.78
2015-10-29 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 105.87, 股數: 923
2015-10-30 📌 賣出成交！ 成交價: 104.34, 股數: -923, 現金剩餘: 98836.10
2015-10-30 🤝 交易平倉：毛利 1405.96, 淨利 1405.96
2

In [14]:
# --- 繪圖 ---
import matplotlib.pyplot as plt

print("\n📈 繪製策略回測圖中 ...")

try:
    fig = cerebro3.plot(
        style='candlestick',
        barup='red',
        bardown='green',
        volume=True,
        iplot=False,  # 使用 matplotlib
        show=False
    )
    plt.show()
    print("📌 圖已顯示")
except Exception as e:
    print(f"⚠️ 無法繪圖: {e}")


📈 繪製策略回測圖中 ...
📌 圖已顯示


In [12]:
# 用來了解 order 生命週期的例子

import backtrader as bt
import yfinance as yf

class OrderLifecycleStrategy(bt.Strategy):
    def __init__(self):
        self.order = None
        self.counter = 0

    def notify_order(self, order):
        dt = self.data.datetime.date(0)
        status = order.getstatusname()

        if order.status in [order.Submitted, order.Accepted]:
            print(f"{dt} 🔄 訂單狀態: {status}")

        elif order.status in [order.Completed]:
            action = "買入" if order.isbuy() else "賣出"
            print(
                f"{dt} 🟢 {action}完成 -> "
                f"價格: {order.executed.price:.2f} 股數: {order.executed.size} "
                f"現金剩餘: {self.broker.getcash():.2f}"
            )
            self.order = None

        else:
            print(f"{dt} 🔴 訂單失敗 -> 狀態: {status}")
            self.order = None

    def notify_trade(self, trade):
        if trade.isclosed:
            dt = self.data.datetime.date(0)
            print(
                f"{dt} 🤝 平倉 -> "
                f"淨利: {trade.pnlcomm:.2f} 現金: {self.broker.getcash():.2f}"
            )
        elif trade.isopen:
            dt = self.data.datetime.date(0)
            print(f"{dt} 🔄 持倉中 -> 價格: {self.data.open[0]:.2f} 股數: {self.position.size}")

    def next(self):
        if self.order:
            return

        self.counter += 1

        # 每 10 根 K 線下個單：買 → 賣 → 買 ...
        if (self.counter % 20) == 1 and not self.position:
            print(f"{self.data.datetime.date(0)} 🚀 發出買進委託")
            cash = self.broker.getcash()
            price = self.data.open[0]  # 若隔天開盤成交
            size = int(cash // price)
            print(f"💰 現金: {cash:.2f} 股價: {price:.2f} 股數: {size}")
            self.order = self.buy(size=size)


        elif (self.counter % 20) == 10 and self.position:
            print(f"{self.data.datetime.date(0)} ⚠️ 發出賣出委託")
            self.order = self.sell(size=self.position.size)




# === 載入測試資料 ===
df = yf.download("AAPL", "2022-01-01", "2022-06-01", auto_adjust=True)
df.columns = df.columns.droplevel(1)
data = bt.feeds.PandasData(dataname=df)

cerebro = bt.Cerebro()
cerebro.addstrategy(OrderLifecycleStrategy)
cerebro.adddata(data)
cerebro.broker.setcash(10000)

print("▶ 回測開始")
cerebro.run()
print("🏁 回測結束")

[*********************100%***********************]  1 of 1 completed

▶ 回測開始
2022-01-03 🚀 發出買進委託
💰 現金: 10000.00 股價: 174.18 股數: 57
2022-01-04 🔄 訂單狀態: Submitted
2022-01-04 🔴 訂單失敗 -> 狀態: Margin
2022-02-01 🚀 發出買進委託
💰 現金: 10000.00 股價: 170.43 股數: 58
2022-02-02 🔄 訂單狀態: Submitted
2022-02-02 🔄 訂單狀態: Accepted
2022-02-02 🟢 買入完成 -> 價格: 171.16 股數: 58 現金剩餘: 72.75
2022-02-02 🔄 持倉中 -> 價格: 171.16 股數: 58
2022-02-14 ⚠️ 發出賣出委託
2022-02-15 🔄 訂單狀態: Submitted
2022-02-15 🔄 訂單狀態: Accepted
2022-02-15 🟢 賣出完成 -> 價格: 167.67 股數: -58 現金剩餘: 9797.64
2022-02-15 🤝 平倉 -> 淨利: -202.36 現金: 9797.64
2022-03-02 🚀 發出買進委託
💰 現金: 9797.64 股價: 161.22 股數: 60
2022-03-03 🔄 訂單狀態: Submitted
2022-03-03 🔴 訂單失敗 -> 狀態: Margin
2022-03-30 🚀 發出買進委託
💰 現金: 9797.64 股價: 175.10 股數: 55
2022-03-31 🔄 訂單狀態: Submitted
2022-03-31 🔄 訂單狀態: Accepted
2022-03-31 🟢 買入完成 -> 價格: 174.41 股數: 55 現金剩餘: 205.20
2022-03-31 🔄 持倉中 -> 價格: 174.41 股數: 55
2022-04-12 ⚠️ 發出賣出委託
2022-04-13 🔄 訂單狀態: Submitted
2022-04-13 🔄 訂單狀態: Accepted
2022-04-13 🟢 賣出完成 -> 價格: 164.16 股數: -55 現金剩餘: 9233.98
2022-04-13 🤝 平倉 -> 淨利: -563.66 現金: 9233.98
2022-04-28 🚀 發出買進委